In [113]:
import numpy as np
import scipy.spatial.distance

In [101]:
class Atom():
    def __init__(self, atomnum, code, resnum, x, y, z):
        self.atomnum = int(atomnum)
        self.code = code
        self.resnum = int(resnum)
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
        
        self.pos = np.array([self.x, self.y, self.z])

In [201]:
#protein_file = './proteins/cln025_trace.pdb'
#protein_file = './proteins/cln025.pdb'
protein_file = './proteins/1lla.pdb'

In [202]:
raw_pdb = open(protein_file)
atom_list = []
for i in raw_pdb.read().split('\n'):
    line_list = i.split()
    if len(line_list) > 0:
        if(line_list[0] == "ATOM"):
            #print(line_list)
            temp_atom = Atom(line_list[1], line_list[2], line_list[5], line_list[6], line_list[7], line_list[8])
            atom_list.append(temp_atom)
raw_pdb.close()

In [203]:
def get_CA_trace(atom_list):
    ca_list = []
    for atom in atom_list:
        if atom.code == 'CA':
            ca_list.append(atom) 
    return(ca_list)

In [208]:
atom_list = get_CA_trace(atom_list)

In [104]:
j= atom_list[0]
k= atom_list[1]
l= atom_list[2]
m= atom_list[3]

In [105]:
m.atomnum, m.code, m.resnum, m.x, m.y, m.z, m.pos

(4, 'CA', 4, -212.11, 18.53, 64.13, array([-212.11,   18.53,   64.13]))

In [123]:
# b1 = np.sqrt(np.sum((j.pos - k.pos)**2))
# b2 = np.sqrt(np.sum((k.pos - l.pos)**2))
# b3 = np.sqrt(np.sum((l.pos - m.pos)**2))
b1 = j.pos - k.pos
b2 = k.pos - l.pos
b3 = l.pos - m.pos

In [120]:
scipy.spatial.distance.euclidean(j.pos,k.pos)

3.906929740857901

In [122]:
b1,b2,b3

(3.9069297408579016, 3.8527133295899398, 3.8523758902786125)

In [124]:
c1md = np.cross(b2, b3)
c2md = np.cross(b1, b2)
print(c1md,c2md)

(array([ 8.5174,  7.8226, -7.8234]), array([-7.5991, -0.5905,  9.2268]))


In [125]:
def compute_cp(u,v):
    out = np.zeros(3)
    out[0] = u[1]*v[2] - u[2]*v[1]
    out[1] = u[2]*v[0] - u[0]*v[2]
    out[2] = u[0]*v[1] - u[1]*v[0]
    return(out)

In [138]:
def compute_dp(u,v):
    out = u[0]*v[0] + u[1]*v[1] + u[2]*v[2]
    return(out)

In [126]:
c1 = compute_cp(b2,b3)
print(c1)

[ 8.5174  7.8226 -7.8234]


In [136]:
scipy.spatial.distance.euclidean(b2,0)

3.85271332958994

In [127]:
c2 = compute_cp(b1,b2)
print(c2)

[-7.5991 -0.5905  9.2268]


In [139]:
np.sqrt(compute_dp(b2,b2))

3.8527133295899398

In [152]:
def compute_dihed(b1,b2,b3):
    c1 = compute_cp(b1,b2)
    c2 = compute_cp(b2,b3)
    temp = compute_cp(c2, c1)
    term1 = compute_dp(temp,b2)/np.sqrt(compute_dp(b2,b2))
    term2 = compute_dp(c2, c1)
    ans = np.arctan2(term1,term2)
    return(ans)

In [156]:
def compute_alpha_angle(a1,a2,a3,a4):
    b1 = a1.pos - a2.pos
    b2 = a2.pos - a3.pos
    b3 = a3.pos - a4.pos
    
    return compute_dihed(b1,b2,b3)

In [153]:
compute_dihed(b1,b2,b3)

-2.5810768984633454

In [161]:
alpha_angles = []
for i in range(1,len(atom_list)-2):
    temp_angle = compute_alpha_angle(atom_list[i-1], atom_list[i], atom_list[i+1], atom_list[i+2])
    alpha_angles.append(temp_angle)

In [162]:
alpha_angles

[-2.5810768984633454,
 -2.8838641280000821,
 1.0289457401802906,
 0.17296066408632557,
 -2.1119247226017563,
 2.6739039783726923,
 -2.576188033424704]

In [180]:
def compute_dist(a1,a2):
    return np.sqrt(np.sum((a1.pos - a2.pos)**2))

In [206]:
import time

In [209]:
t0 = time.time()
distances = []
for a1 in atom_list[:-3]:
    for a2 in atom_list[a1.resnum+2:]:
        distances.append(compute_dist(a1,a2)/10.)
        #print(a1.resnum, a2.resnum)
t1 = time.time()
print(t1-t0)

1.35456109047


In [210]:
1.35456109047*1/60*1/60*100000

37.6266969575

In [183]:
distances = []
for a1 in atom_list[:-3]:
    for a2 in atom_list[a1.resnum+2:]:
        distances.append(compute_dist(a1,a2)/10.)
        #print(a1.resnum, a2.resnum)
print(distances)

[0.99265855156745497, 1.2427972481462928, 1.2173697055537409, 1.0143421513473645, 0.84055338914312894, 0.57230149396974306, 0.49548965680425711, 0.90569641712882965, 0.88633740753733348, 0.65638631917491985, 0.59464022736441269, 0.45483843285280878, 0.62716584728443237, 0.58372082368200728, 0.51721658906110224, 0.52876554350676241, 0.63281750923943225, 0.79713298765011631, 0.53135016702735716, 0.76610769477926566, 0.91804901829912988, 1.1572972824646215, 0.81798105112526898, 1.0840239849744999, 1.2930533631679697, 0.88749760563057278, 1.1297021731412227, 0.96985050394377725]


In [190]:
zip(distances, msmb_contacts[0][0])

[(0.99265855156745497, 0.99265856),
 (1.2427972481462928, 1.2427971),
 (1.2173697055537409, 1.2173699),
 (1.0143421513473645, 1.0143422),
 (0.84055338914312894, 0.84055322),
 (0.57230149396974306, 0.57230127),
 (0.49548965680425711, 0.49548906),
 (0.90569641712882965, 0.90569609),
 (0.88633740753733348, 0.88633758),
 (0.65638631917491985, 0.65638632),
 (0.59464022736441269, 0.59464025),
 (0.45483843285280878, 0.45483851),
 (0.62716584728443237, 0.62716573),
 (0.58372082368200728, 0.58372086),
 (0.51721658906110224, 0.51721591),
 (0.52876554350676241, 0.52876502),
 (0.63281750923943225, 0.63281673),
 (0.79713298765011631, 0.79713213),
 (0.53135016702735716, 0.53134918),
 (0.76610769477926566, 0.7661072),
 (0.91804901829912988, 0.9180485),
 (1.1572972824646215, 1.1572967),
 (0.81798105112526898, 0.81798035),
 (1.0840239849744999, 1.0840232),
 (1.2930533631679697, 1.2930526),
 (0.88749760563057278, 0.88749778),
 (1.1297021731412227, 1.1297022),
 (0.96985050394377725, 0.9698506)]

In [191]:
assert np.allclose(distances, msmb_contacts[0][0])

In [192]:
np.allclose??

In [167]:
import pandas as pd

In [164]:
import mdtraj as md
from msmbuilder.featurizer import AlphaAngleFeaturizer
from msmbuilder.featurizer import ContactFeaturizer

In [211]:
t = md.load(protein_file)

In [146]:
feat = AlphaAngleFeaturizer(sincos=False)

In [151]:
feat.fit_transform(t)

[array([[-2.58107948, -2.88386488,  1.02894557,  0.17296059, -2.11192536,
          2.67390347, -2.57618546]], dtype=float32)]

In [212]:
con = ContactFeaturizer(scheme='CA')

In [213]:
t2 = time.time()
msmb_contacts = con.fit_transform(t)
t3 = time.time()
print(t3-t2)

3.0936460495


In [169]:
df = con.describe_features(t)

In [173]:
pd.DataFrame(df)

,atominds,featuregroup,featurizer,otherinfo,resids,resnames,resseqs
0,N/A,CA,Contact,Ignore_Protein True,"[0, 3]","[TYR, PRO]","[1, 4]"
1,N/A,CA,Contact,Ignore_Protein True,"[0, 4]","[TYR, GLU]","[1, 5]"
2,N/A,CA,Contact,Ignore_Protein True,"[0, 5]","[TYR, THR]","[1, 6]"
3,N/A,CA,Contact,Ignore_Protein True,"[0, 6]","[TYR, GLY]","[1, 7]"
4,N/A,CA,Contact,Ignore_Protein True,"[0, 7]","[TYR, THR]","[1, 8]"
5,N/A,CA,Contact,Ignore_Protein True,"[0, 8]","[TYR, TRP]","[1, 9]"
6,N/A,CA,Contact,Ignore_Protein True,"[0, 9]","[TYR, TYR]","[1, 10]"
7,N/A,CA,Contact,Ignore_Protein True,"[1, 4]","[TYR, GLU]","[2, 5]"
8,N/A,CA,Contact,Ignore_Protein True,"[1, 5]","[TYR, THR]","[2, 6]"
9,N/A,CA,Contact,Ignore_Protein True,"[1, 6]","[TYR, GLY]","[2, 7]"


In [ ]:
#         ca = [a.index for a in traj.top.atoms if a.name == 'CA']
#         if len(ca) < 4:
#             return np.zeros((len(traj), 0), dtype=np.float32)

#         alpha_indices = np.array(
#             [(ca[i - 1], ca[i], ca[i + 1], ca[i + 2]) for i in range(1, len(ca) - 2)])
#         result = md.compute_dihedrals(traj, alpha_indices)

In [ ]:
# def _dihedral(traj, indices, periodic, out=None):
#     """Compute the dihedral angles of traj for the atom indices in indices.
#     Parameters
#     ----------
#     xyz : np.ndarray, shape=(num_frames, num_atoms, 3), dtype=float
#         The XYZ coordinates of a trajectory
#     indices : np.ndarray, shape=(num_dihedrals, 4), dtype=int
#         Atom indices to compute dihedrals.
#     periodic : bool, default=True
#         If `periodic` is True and the trajectory contains unitcell
#         information, we will treat dihedrals that cross periodic images
#         using the minimum image convention.
#     Returns
#     -------
#     dih : np.ndarray, shape=(num_dihedrals), dtype=float
#         dih[i,j] gives the dihedral angle at traj[i] correponding to indices[j].
#     """
#     ix10 = indices[:, [0, 1]]
#     ix21 = indices[:, [1, 2]]
#     ix32 = indices[:, [2, 3]]

#     b1 = distance.compute_displacements(traj, ix10, periodic=periodic, opt=False)
#     b2 = distance.compute_displacements(traj, ix21, periodic=periodic, opt=False)
#     b3 = distance.compute_displacements(traj, ix32, periodic=periodic, opt=False)

#     c1 = np.cross(b2, b3)
#     c2 = np.cross(b1, b2)

#     p1 = (b1 * c1).sum(-1)
#     p1 *= (b2 * b2).sum(-1) ** 0.5
#     p2 = (c1 * c2).sum(-1)

#     return np.arctan2(p1, p2, out)